In [82]:
import pandas as pd
import numpy as np

In [ ]:
donnees_mes = pd.read_excel("../data/donnees_mesures_PM2_5.xlsx")
donnees_env = pd.read_excel("../data/donnees_environnement_capteurs.xlsx")
donnees_mes

In [ ]:
date = donnees_mes['Date']
donnees_mes = donnees_mes.drop(donnees_mes.columns[0], axis=1)
# Remplacement des virgules par des points et conversion en float pour toutes les colonnes
for nom_colonne in donnees_mes.columns:
    donnees_mes[nom_colonne] = donnees_mes[nom_colonne].replace(',', '.').astype(float)

In [ ]:
# [['c7', 37.93437790878064], ['c26', 46.95547626434998], ['c27', 41.79336022339435]] with NaN before aberrant

In [ ]:
#Suppression des valeurs aberrantes

donnees_mes.hist()
    
for colonne in donnees_mes.columns:
        if pd.api.types.is_numeric_dtype(donnees_mes[colonne]):
            Q1 = donnees_mes[colonne].quantile(0.25)
            Q3 = donnees_mes[colonne].quantile(0.75)
            IQR = Q3 - Q1
            
            # Détection des valeurs aberrantes
            outliers_mask = (donnees_mes[colonne] < (Q1 - 1.5 * IQR)) | (donnees_mes[colonne] > (Q3 + 1.5 * IQR))
            
            # Remplacement des valeurs aberrantes par NaN
            donnees_mes.loc[outliers_mask, colonne] = np.nan
        


In [ ]:
# Supression des colonnes aberrantes et remplissage des valeurs nulles

# Compter le nombre de valeurs manquantes par colonne
nombre_na_par_colonne = donnees_mes.isna().sum()

# Supprimer les colonnes avec un pourcentage de valeurs manquantes supérieur à 33%
colonnes_del = []
for nom_colonne, nombre_na in nombre_na_par_colonne.items():
    if nombre_na / len(donnees_mes) > 0.33:
        colonnes_del.append([nom_colonne,nombre_na*100/len(donnees_mes)])
        donnees_mes = donnees_mes.drop(nom_colonne, axis=1)
        

# Remplir les valeurs manquantes restantes avec la médiane de chaque colonne
donnees_mes = donnees_mes.fillna(donnees_mes.median())

print(colonnes_del)
donnees_mes.hist()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalisation des données

scaler = StandardScaler()
X = donnees_mes.to_numpy()
X = scaler.fit_transform(X)
mes_normees = pd.DataFrame(X)
mes_normees


In [ ]:
mes_normees.insert(0, 'Date', date)

In [ ]:
mes_normees.to_csv("../preprocessing/donnees_mesures_PM2_5.csv",index=False,sep=';')